In [37]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import os
import re
from PyPDF2 import PdfReader
from transformers import AutoModel, AutoTokenizer

In [2]:
# model_name = 'DeepPavlov/rubert-base-cased-sentence'
model_name = 'DeepPavlov/bert-base-multilingual-cased-sentence'
# model_name = 'DeepPavlov/distilrubert-small-cased-conversational'
# model_name = 'DeepPavlov/distilrubert-tiny-cased-conversational'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
length_thr = 100
device = torch.device(0)
# device = 'cpu'
encode_kwargs = {'truncation': True, 'padding': 'max_length', 'pad_to_multiple_of': 1, 'max_length':512}

In [56]:
def get_abstract(text):
    if ('abstract' in text.lower()) and ('ntroduction' in text.lower()):
        start_pos = text.lower().index('abstract') + len('abstract')
        end_pos = text.lower().index('ntroduction') - 1
        abstract = text[start_pos: end_pos]
    else:
        abstract = text

    abstract = re.sub('-\n', '', abstract)
    abstract = re.sub('\n', ' ', abstract)
    return abstract

In [101]:
db_path = '/home/booydar/Documents/Sync/zotero/storage/'
path = db_path
def parse_storage(path):

    db_df = pd.DataFrame()
    path, folders, files = next(os.walk(path))
    for f in folders:
        f_path = os.path.join(path, f)
        path_, folders_, files_ = next(os.walk(f_path))
        pdfs = [n for n in files_ if '.pdf' in n]
        if not pdfs:
            continue
        paper = pdfs[0]
        paper_path = os.path.join(f_path, paper)
        reader = PdfReader(paper_path)
        page = reader.pages[0]
        first_page = page.extract_text()
        abstract = get_abstract(first_page)

        paper_dict = {'path': paper_path,
                    'paper': paper,
                    'name': re.sub(r'.*-* - ', '', paper)[:-4], 
                    'abstract': abstract}

        db_df = pd.concat([db_df, pd.DataFrame(paper_dict, index=[0])])
        
    return db_df

In [118]:
db_df = parse_storage(db_path)

In [119]:
out_path = '/home/booydar/Documents/Sync/scientific_graph'

for i, row in db_df.iterrows():
    
    note = ''
    note += '#paper\n---\n\n'
    note += f"##{row['name']}\n\n"
    note += f'{row.abstract}\n\n---\n'

    note_name = row.paper[:-4] + '.md'
    op = os.path.join(out_path, note_name)

    with open(op, 'w') as f:
        f.write(note)